In [4]:
!ls /data/

Player.csv  test


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,avg
import time

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PySpark Example with Sleep") \
    .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 19:31:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Create a sample DataFrame
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29), ("David", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()

# Add a new column with age incremented by 1
df_with_new_col = df.withColumn("AgePlusOne", col("Age") + 1)

# Show the DataFrame with the new column
df_with_new_col.show()

# Calculate the average age
average_age = df.agg(avg("Age")).collect()[0][0]
print(f"The average age is: {average_age}")

# Filter the DataFrame to include only rows where age is greater than 30
filtered_df = df.filter(col("Age") > 30)

# Show the filtered DataFrame
filtered_df.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
|David| 35|
+-----+---+

+-----+---+----------+
| Name|Age|AgePlusOne|
+-----+---+----------+
|Alice| 34|        35|
|  Bob| 45|        46|
|Cathy| 29|        30|
|David| 35|        36|
+-----+---+----------+

The average age is: 35.75
+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|David| 35|
+-----+---+



In [7]:
df = spark.read.csv("/data/Player.csv",header=True)
df.show()

+---+-------------+--------------------+------------------+-------------------+------+------+
| id|player_api_id|         player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+--------------------+------------------+-------------------+------+------+
|  1|       505942|  Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|     Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|         Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|       Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|        Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
|  6|        27316|          Aaron Hunt|            158138|1986-09-04 00:00:00|182.88|   161|
|  7|       564793|          Aaron Kuhl|            221280|1996-01-30 00:00:00|172.72|   146|
|  8|        30895|        Aaron Lennon|            152747|1